In [1]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.ai.projects.models import FilePurpose
from azure.identity import DefaultAzureCredential
from pathlib import Path

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

with project_client:
    # upload a file and wait for it to be processed
    file = project_client.agents.upload_file_and_poll(
        file_path="../../data/nifty_500_quarterly_results.csv", purpose=FilePurpose.AGENTS
    )
    print(f"Uploaded file, file ID: {file.id}")

    # instantiate the CodeInterpreterTool with the necessary files
    code_interpreter = CodeInterpreterTool(file_ids=[file.id])

    # notice that CodeInterpreter must be enabled in the agent creation, otherwise the agent will not be able to see the file attachment
    agent = project_client.agents.create_agent(
        model="gpt-4o-mini",
        name="my-agent",
        instructions="You are helpful agent",
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    print(f"Created agent, agent ID: {agent.id}")

    # create a thread
    thread = project_client.agents.create_thread()
    print(f"Created thread, thread ID: {thread.id}")

    # create a message
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Could you please create bar chart in the TRANSPORTATION sector for the operating profit from the uploaded csv file and provide file to me?",
    )
    print(f"Created message, message ID: {message.id}")

    # create and execute a run
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # delete the original file from the agent to free up space (note: this does not delete your version of the file)
    project_client.agents.delete_file(file.id)
    print("Deleted file")

    # print the messages from the agent
    messages = project_client.agents.get_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    # get the most recent message from the assistant
    last_msg = messages.get_last_text_message_by_sender("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

    # save the newly created file
    for image_content in messages.image_contents:
        print(f"Image File ID: {image_content.image_file.file_id}")
        file_name = f"{image_content.image_file.file_id}_image_file.png"
        project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
        print(f"Saved image file to: {Path.cwd() / file_name}")
    
    # iterates through file_path_annotations in messages and prints details for each annotation
    for file_path_annotation in messages.file_path_annotations:
        print(f"File Paths:")
        print(f"Type: {file_path_annotation.type}")
        print(f"Text: {file_path_annotation.text}")
        print(f"File ID: {file_path_annotation.file_path.file_id}")
        print(f"Start Index: {file_path_annotation.start_index}")
        print(f"End Index: {file_path_annotation.end_index}")

    # delete the agent once done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

Uploaded file, file ID: assistant-ILaDBwm42chCuxUUaZniFehb
Created agent, agent ID: asst_ZqrbwUqdxRjSy9IUIkk06zQj
Created thread, thread ID: thread_gpaAgXC3ZnY713AhyAxzfY9d
Created message, message ID: msg_iWIQPzj9VewzGmvnSeQ4YGUT
Run finished with status: RunStatus.COMPLETED
Deleted file
Messages: <azure.ai.projects.models._patch.ThreadMessages object at 0x00000249B4308110>
Last Message: The bar chart displaying the operating profit in the Transportation sector has been created successfully. You can download it using the link below:

[Download the bar chart](sandbox:/mnt/data/transportation_operating_profit_chart.png)
Image File ID: assistant-Y7bTy3sFd4VzEkNukh3F8UZg
Saved image file to: d:\azure-ai-agents\samples\code-interpreter\assistant-Y7bTy3sFd4VzEkNukh3F8UZg_image_file.png
File Paths:
Type: file_path
Text: sandbox:/mnt/data/transportation_operating_profit_chart.png
File ID: assistant-VRDDcK6ltcELRcjJi4CVMl7m
Start Index: 174
End Index: 233
Deleted agent
